In [ ]:
import numpy as np
import elboflow as ef
import tensorflow as tf
import scipy.stats
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
import logging

%matplotlib inline

In [ ]:
# Generate a network with different communities
np.random.seed(2)
num_nodes = 100
num_communities = 5

# Generate parameters to infer
z = np.random.randint(num_communities, size=num_nodes)
connection_probas = np.random.uniform(0, .1, (num_communities, num_communities))
connection_probas += np.eye(num_communities) * np.random.uniform(0.4, 0.8, num_communities)
np.testing.assert_array_less(connection_probas, 1)

# Get the actual connection probabilities
proba = connection_probas[z[:, None], z[None, :]]
adjacency = np.random.uniform(0, 1, proba.shape) < proba

# Create a one hot encoding
onehot = np.zeros((num_nodes, num_communities))
onehot[np.arange(num_nodes), z] = 1

In [ ]:
try:
    import networkx as nx
    graph = nx.from_numpy_matrix(adjacency, create_using=nx.DiGraph())
    
    fig, ax = plt.subplots(1, 1)
    pos = nx.spectral_layout(graph)
    nx.draw_networkx_nodes(graph, pos, node_color=z, cmap='Vega10')
    nx.draw_networkx_edges(graph, pos)
    ax.set_aspect(1)
    
except ImportError:
    logging.warning("this example requires the package `networkx` for visualization")

In [ ]:
class StochasticBlockModel(ef.Model):
    def __init__(self, adjacency, num_communities, create_optimizer=None, create_session=None):
        self.num_communities = num_communities
        super(StochasticBlockModel, self).__init__(None, [adjacency], create_optimizer, create_session)
        
    def setup(self, adjacency):
        n, _ = adjacency.shape
        adjacency = ef.as_tensor(adjacency)
        
        # Create the distributions
        with tf.variable_scope('z', initializer=tf.random_normal_initializer(stddev=0.01)): 
            q_z = ef.CategoricalDistribution(
                ef.get_normalized_variable('z_p', (n, self.num_communities))
            )
            
        with tf.variable_scope('proba', initializer=tf.random_normal_initializer(np.log(np.exp(1) - 1), 0.01)):
            q_proba = ef.BetaDistribution(
                ef.get_positive_variable('proba_a', (self.num_communities, self.num_communities)),
                ef.get_positive_variable('proba_b', (self.num_communities, self.num_communities))
            )
        
        # Evaluate the log-joint distribution
        proba_log = ef.evaluate_statistic(q_proba, 'log')
        proba_log1m = ef.evaluate_statistic(q_proba, 'log1m')
        adjacency = adjacency[..., None, None]
        # Get the likelihood for all edges for all possible combinations
        bernoulli_ll = tf.add(adjacency * proba_log, (1.0 - adjacency) * proba_log1m, 'pointwise_ll')
        mixture_ll = ef.CategoricalDistribution.interacting_mixture_log_likelihood(q_z, bernoulli_ll, 'pointwise_mixture_ll')

        return tf.reduce_sum(mixture_ll), {
            'z': q_z,
            'proba': q_proba
        }, {
            'z': ef.CategoricalDistribution(np.ones((1, self.num_communities), np.float32) / self.num_communities).log_proba,
            'proba': ef.BetaDistribution(1.0, 1.0).log_proba
        }
    
    def create_optimizer(self):
        self.lr = tf.where(self.global_step < 500, 0.01, 0.1)
        return tf.train.GradientDescentOptimizer(self.lr)
    
model = StochasticBlockModel(adjacency, num_communities)
model.run(model.loss)

In [ ]:
# Show the initial conditions for the community assignments
fig, (ax1, ax2) = plt.subplots(1, 2)
im = ax1.imshow(model.run(model['z'].statistic(1)).T, aspect='auto')
plt.colorbar(im, ax=ax1)
ax1.set_ylabel('Communities')
ax1.set_xlabel('Nodes')
ax1.set_title('$P(z)$')

ax2.plot(np.log(num_communities) - model.run(model['z'].entropy))
ax2.set_xlabel('Nodes')
ax2.set_ylabel('max entropy - entropy')

fig.tight_layout()

# Show the initial conditions for the connection probabilities
fig, (ax1, ax2) = plt.subplots(1, 2)
model.plot_proba('proba', ax=ax1)
ax1.set_ylabel(r'$P(\rho)$')
ax1.set_xlabel(r'Connection probability $\rho$')

ax2.plot(0 - model.run(model['proba'].entropy).ravel())
ax2.set_xlabel(r'Community combinations')
ax2.set_ylabel('max entropy - entropy')
fig.tight_layout()

In [ ]:
trace = model.optimize(2000, model.loss, tqdm=tqdm_notebook)
plt.plot(trace[model.loss])
plt.yscale('log')
print('Initial loss: %f' % trace[model.loss][0])
print('Final   loss: %f' % trace[model.loss][-1])

In [ ]:
plt.imshow(model.run(model['z'].statistic(1))[np.argsort(z)], aspect='auto')

In [ ]:
z_estimate = np.argmax(model.run(model['z'].statistic(1)), axis=1)
try:
    import networkx as nx
    fig, ax = plt.subplots(1, 1)
    nx.draw_networkx_nodes(graph, pos, node_color=z_estimate, cmap='Vega10')
    nx.draw_networkx_edges(graph, pos)
    ax.set_aspect(1)
    
except ImportError:
    logging.warning("this example requires the package `networkx` for visualization")